In [2]:
pwd

'/mmfs1/data/bergerar/BC-MRI-ADHD/Code'

In [3]:
import os 
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from make_models import get_MRI_VAE_3D,get_MRI_CVAE_3D
from tqdm import tqdm 

2022-04-05 10:03:32.313652: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/local/apps/gcc/9.2.0/lib:/cm/local/apps/gcc/9.2.0/lib64:/cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2022-04-05 10:03:32.313691: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
data_stuff = '../Assets/brain_array-440.npz'

In [5]:
class bergeronADHD200_data_loader():
    
    def __init__(self,data_dir,batch_size=32):

        self.data = np.load(data_stuff)['data']
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.n_batches = int(self.data.shape[0]/self.batch_size)-1
        self.e = -1
        self.b = -1
        
        self.new_epoch()
        
    def new_epoch(self):
        self.e+=1
        self.perm_idx = np.random.permutation(np.arange(self.data.shape[0]))
        self.data = self.data[self.perm_idx,:,:,:]
        
        
# figure out how to differentiate between patients and controls to separate into two batches 

    def get_patient_batch(self):
        
        if self.b==self.n_batches:
            self.new_epoch()
            self.b = -1
                    
        self.b+=1        
        self.batch_idx = np.arange(self.b*self.batch_size,self.b*self.batch_size+self.batch_size)
        self.batch = self.data[self.batch_idx,:,:,:]
                    
        return self.batch
    
    def get_patient_batch(self):
        
        if self.b==self.n_batches:
            self.new_epoch()
            self.b = -1
                    
        self.b+=1        
        self.batch_idx = np.arange(self.b*self.batch_size,self.b*self.batch_size+self.batch_size)
        self.batch = self.data[self.batch_idx,:,:,:]
                    
        return self.batch

In [6]:
loaderhehe = bergeronADHD200_data_loader(data_dir=data_stuff,batch_size=32)

In [7]:
loaderhehe.data.shape

(440, 64, 64, 64)

In [29]:
data = data_stuff['data']
control = data_stuff['controls']
patients = data_stuff['patients']
#how do I determine which of the 440 are patients/ put them in order so its like the first half is patients second half controls so I can select half the data

TypeError: string indices must be integers

In [21]:
n = loaderhehe.data.shape[0]
print(n)

440


In [26]:
batch_size = 32

In [32]:
data_batch = data_stuff[np.random.randint(low=0,high=n,size=batch_size)]
# weird error

TypeError: only integer scalar arrays can be converted to a scalar index

In [28]:
data_batch_patients = patients[np.random.randint(low=0,high=n,size=batch_size),:,:,:]
data_batch_control = control[np.random.randint(low=0,high=n,size=batch_size),:,:,:]

NameError: name 'patients' is not defined

In [ ]:
data_batch_control = data_stuff[np.random.randint(low=0,high=n,size=batch_size),:,:,:]

In [30]:
loaderhehe.get_patient_batch();
loaderhehe.get_control_batch();

AttributeError: 'bergeronADHD200_data_loader' object has no attribute 'get_patient_batch'

In [ ]:
cvae, z_encoder, s_encoder, cvae_decoder = get_MRI_CVAE_3D(input_shape=(64,64,64,1),
                                            latent_dim=2,
                                            beta=1,
                                            disentangle=False,
                                            gamma=1,
                                            bias=True,
                                            batch_size = 64,
                                            kernel_size = 3,
                                            filters = 32,
                                            intermediate_dim = 128,
                                            opt=None)